### Importazione delle librerie

In [ ]:
import pathlib
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

### Definizione delle variabili
Scelta del numero di classi (Healthy e Patient) e numero di immagini sintetiche da generare

In [1]:
num_classes = 2
synth_images = 3

### Seleziono la classe per la quale generare immagini:

0 -> [0. 1.] -> Healthy

1 -> [1. 0.] -> Patient

In [ ]:

healthy_or_patient = 1
# Per aggiungere la classe al percorso e al nome del file
if healthy_or_patient == 1:
    healthy_or_patient_type = "Patient ([0. 1.])"
elif healthy_or_patient == 0:
    healthy_or_patient_type = "Healthy ([1. 0.])"

### Seleziono uno dei modelli salvati

In [ ]:
good_checkpoint_20220521 = "best_augmented_generator_20220521-162822_cp-epo0100-g_loss1.65-d_loss0.34-acc0.87-mse0.09.h5"
meander02_40epo = "best_not_augmented_generator_20220507-145052_cp-epo0040-g_loss0.78-d_loss0.72-acc0.59-mse0.24.h5"
meander02_epo500 = "cgan_generator_meander02_epoch501.h5"
meander02_epo1000 = "cgan_gen_meander02_epoch1001_20220508-010316.h5"

model_name = good_checkpoint_20220521

### Carico il modello del generatore.


In [ ]:
trained_generator = tf.keras.models.load_model("models/testing/" + model_name)
trained_generator.summary()

In [ ]:
generated_images_dir = "logs/generated_images/from_best_models/"
model_name_dir = pathlib.Path(generated_images_dir, model_name)
model_name_dir.mkdir(parents=True, exist_ok=True)
healthy_or_patient_dir = pathlib.Path(model_name_dir, healthy_or_patient_type)
healthy_or_patient_dir.mkdir(parents=True, exist_ok=True)

### Modifico lo script di default eliminando l'interpolazione e generando immagini al posto della gif
Riferimento: https://keras.io/examples/generative/conditional_gan/

In [ ]:
latent_dim = 128
# Sample noise
noise = tf.random.normal(shape=(1, latent_dim))
noise = tf.repeat(noise, repeats=synth_images)
noise = tf.reshape(noise, (synth_images, latent_dim))
print(noise.shape)

def generate_images_and_labels(class_number):
    # Convert the start and end labels to one-hot encoded vectors.
    class_label = tf.keras.utils.to_categorical([class_number], num_classes)
    print(f'Class label to_categorical: ', class_label)
    class_label = tf.cast(class_label, tf.float32)
    print(f'Class label cast: ', class_label)
   
    print(f'Class label shape before: ', class_label.shape)
    print(class_label)
    class_label = tf.repeat(class_label, synth_images, axis=0)
    print(f'Class label shape after: ', class_label.shape)
    print(class_label)
    
    # Combine the noise and the labels and run inference with the generator.
    noise_and_labels = tf.concat([noise, class_label], 1)
    synth_img = trained_generator.predict(noise_and_labels)
    return synth_img, class_label

fake_images, label = generate_images_and_labels(healthy_or_patient)

In [ ]:

synth_images *= 255.0
generated_images = fake_images.astype(np.uint8)
for i in range(synth_images):
    img = tf.keras.preprocessing.image.array_to_img(generated_images[i])
    img.save(f"{healthy_or_patient_dir}/synt_{healthy_or_patient_type}_{i+1}_{model_name}.png")
    print("Salvata immagine con nome synt_{}_{}_{}.png".format(healthy_or_patient_type, i+1, model_name))
    

### Visualizzo un esempio di immagini generate

In [ ]:
print(f'Nome del modello: ', model_name)
print(f'Classe delle immagini sintetiche: ', healthy_or_patient_type)
plt.figure(figsize=(12, 12))
for i in range(synth_images):
    plt.subplot(1, 5, i + 1)
    plt.imshow(generated_images[i], cmap='gray')
    plt.title((label[i].numpy()))
    plt.axis('off')